In [ ]:
import cv2
import pytesseract
import numpy as np
from pytesseract import Output
from PIL import Image
from rembg import remove
from matplotlib import pyplot as plt

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 39.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Installation des packages nécessaires (décommentez si besoin)
!pip install pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-fra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-fra
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 527 kB of archives.
After this operation, 1,145 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-fra all 1:4.00~git30-7274cfa-1.1 [527 kB]
Fetched 527 kB in 1s (526 kB/s)
Selecting previously unselected package tesseract-ocr-fra.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-fra_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-fra (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-fr

In [ ]:
from PIL import Image, ImageEnhance
import pytesseract

# Charger l'image
image_path = 'notice.jpg'
image = Image.open(image_path)

# Diviser l'image en deux parties verticales
largeur, hauteur = image.size
moitie_gauche = image.crop((0, 0, largeur // 2, hauteur))
moitie_droite = image.crop((largeur // 2, 0, largeur, hauteur))

# Prétraitement pour la moitié gauche
moitie_gauche_gray = moitie_gauche.convert('L')
enhancer_gauche = ImageEnhance.Contrast(moitie_gauche_gray)
moitie_gauche_enhanced = enhancer_gauche.enhance(2)

# Prétraitement pour la moitié droite
moitie_droite_gray = moitie_droite.convert('L')
enhancer_droite = ImageEnhance.Contrast(moitie_droite_gray)
moitie_droite_enhanced = enhancer_droite.enhance(2)

# Option pour améliorer la reconnaissance
custom_config = r'--oem 3 --psm 6'

# OCR sur la moitié gauche
text_gauche = pytesseract.image_to_string(moitie_gauche_enhanced, lang='fra', config=custom_config)

# OCR sur la moitié droite
text_droite = pytesseract.image_to_string(moitie_droite_enhanced, lang='fra', config=custom_config)

# Afficher le texte extrait des deux moitiés
print("TEXTE DE LA COLONNE GAUCHE:")
print("--------------------------")
print(text_gauche)
print("\nTEXTE DE LA COLONNE DROITE:")
print("--------------------------")
print(text_droite)

# Sauvegarder le texte dans un fichier (optionnel)
with open('texte_extrait.txt', 'w', encoding='utf-8') as f:
    f.write("TEXTE DE LA COLONNE GAUCHE:\n")
    f.write("--------------------------\n")
    f.write(text_gauche)
    f.write("\n\nTEXTE DE LA COLONNE DROITE:\n")
    f.write("--------------------------\n")
    f.write(text_droite)

TEXTE DE LA COLONNE GAUCHE:
--------------------------
GG ;
=
Duphaston 10 mg, comprhépalicuié
Dydrogestérone y, è

Veuillez lire attentivement cette notice avant de prendre ce médicament | ,

car elle contient des informations importantes pour vous. Fr »

- Gardez cette notice. Vous pourriez avoir besoin de la relire. |

- Si vous avez d'autres questions, interrogez votre médecin ou votre
pharmacien.

- Ce médicament vous a été personnellement prescrit. Ne le donnez
pas à d'autres personnes. || pourrait leur être nocif, même si les
signes de leur maladie sont identiques aux vôtres.

- Si vous ressentez un quelconque effet indésirable, parlez-en à
votre médecin ou votre pharmacien. Ceci s'applique aussi à tout
effet indésirable qui ne serait pas mentionné dans cette notice. Voir
rubrique 4.

Que contient cette notice
1. Qu'est-ce que Duphaston 10 mg, comprimé pelliculé ‘et dans quels
cas est-il utilisé ?
2. Quelles sont les informations à connaître avant de prendre ce
médicament ?
3. C

In [ ]:
from transformers import pipeline, AutoTokenizer
import re
import json

MODEL_SUMMARY = "facebook/bart-large-cnn"  # Plus robuste pour le général
MAX_TOKENS = 1024

tokenizer = AutoTokenizer.from_pretrained(MODEL_SUMMARY)
summarizer = pipeline("summarization", model=MODEL_SUMMARY, tokenizer=MODEL_SUMMARY)

def nettoyer_texte(texte):
    corrections = [
        (r'\s+', ' '),
        (r'(\d)\s?([mgµ]?g|UI|ml)', r'\1\2'),
        (r'(\w)-\s', r'\1-'),
        (r'TEXTE DE LA COLONNE [A-Z]+:[\s\-]*', ''),
        (r'\x0c|\u2028|\u200b', ' ')
    ]
    for pattern, repl in corrections:
        texte = re.sub(pattern, repl, texte, flags=re.IGNORECASE)
    texte = re.sub(r'(actualité|infos|scoop|automobile)[^\.!?]*[\.!?]', '', texte, flags=re.IGNORECASE)  # Supprime les titres hors sujet
    return texte.strip()

def segmenter_notice(texte):
    # Repérage multi-lignes avec titres en majuscules ou soulignés
    titres = [
        'COMPOSITION', 'INDICATIONS', 'POSOLOGIE', 'CONTRE-INDICATIONS', 'EFFETS SECONDAIRES', 'INTERACTIONS'
    ]
    pattern = r'(' + '|'.join(titres) + r')\s*:?'
    matches = list(re.finditer(pattern, texte, re.IGNORECASE))
    sections = {}
    for i, match in enumerate(matches):
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(texte)
        titre = match.group(1).upper()
        contenu = texte[start:end].strip()
        if contenu:
            sections[titre] = contenu
    return sections

def decouper_texte(texte, max_tokens=MAX_TOKENS):
    mots = texte.split()
    chunks = []
    chunk = []
    total_tokens = 0
    for mot in mots:
        tokens = tokenizer(mot, return_tensors="pt", add_special_tokens=False).input_ids.shape[1]
        if total_tokens + tokens > max_tokens - 10:
            chunks.append(' '.join(chunk))
            chunk = [mot]
            total_tokens = tokens
        else:
            chunk.append(mot)
            total_tokens += tokens
    if chunk:
        chunks.append(' '.join(chunk))
    return chunks

def generer_resume(texte):
    if not texte or len(texte.split()) < 30:
        return texte  # Trop court pour résumer
    chunks = decouper_texte(texte)
    resumes = []
    for chunk in chunks:
        min_len = min(30, len(chunk)//4)
        max_len = min(150, len(chunk)//2)
        try:
            sortie = summarizer(chunk, max_length=max_len, min_length=min_len, truncation=True)
            resumes.append(sortie[0]['summary_text'])
        except Exception as e:
            resumes.append(chunk)
    return " ".join(resumes)

# Chargement et nettoyage du texte
with open('texte_extrait.txt', 'r', encoding='utf-8') as f:
    texte_brut = f.read()
texte_propre = nettoyer_texte(texte_brut)

# Résumé global
resume_global = generer_resume(texte_propre)

# Extraction et résumé des sections
sections = segmenter_notice(texte_propre)
resumes_par_section = {}
for section, contenu in sections.items():
    resumes_par_section[section] = generer_resume(contenu)

# Export JSON
with open('resume_medical.json', 'w', encoding='utf-8') as f:
    json.dump({'Resume_Global': resume_global, 'Sections': resumes_par_section}, f, ensure_ascii=False, indent=2)

print("Résumé global généré:")
print(resume_global)
print("\nPar section:")
for section, resume in resumes_par_section.items():
    print(f"\n{section}:")
    print(resume)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Résumé global généré:
Dydrogestèrone est une hormone progestagène féminine. Elle régule avec les hormones œstrogènes le cycle menstruel. Vous pourriez avoir besoin de la relire. Pour certaines femmes utilisant un traitement hormonal substitutif (THS), prendre seulement un œstrogène peut provoquer un épaississement anormal of la muqueuse utérine. Plus de 9 millions of femmes enceintes ont pris Duphaston 10mg, comEnné pelliculé. Vous pouvez prendre les comprimés avec ou sans nourriture. La ligne de sécabilité du comprimé. n'est là que pour faciliter la prise duComprimé en facilitant la division. Elle ne le divise pas en deux | demi-doses égales.

Par section:

INDICATIONS:
Vous pouvez prendre les comprimés avec ou sans nourriture. Vérifiez auprès de votre médecin ou pharmacien.

POSOLOGIE:
La journée de début et le nombre of jours pendant lesquels vous prenez vos comprimés dépendront de la durée of votre cycle. Si vous n'avez pas de cycles menstruels naturels, votre médecin vous recomman